In [0]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F

dbutils.fs.cp("file:/Workspace/Shared/product_data.json", "dbfs:/FileStore/product_data.json")

dbutils.fs.cp("file:/Workspace/Shared/employee_data.csv", "dbfs:/FileStore/employee_data.csv")

product_df = spark.read.option("multiline", "true").json("/FileStore/product_data.json") 

employee_df = spark.read.format("csv").option("header", "true").load("/FileStore/employee_data.csv")

# Converting CSV and JSON to delta format

employee_df.write.format("delta").mode("overwrite").save("/Workspace/Shared/employee_table")

product_df.write.format("delta").mode("overwrite").save("/Workspace/Shared/product_table")

# Register Delta Tables

# employee_df.write.saveAsTable("employee_table")
# product_df.write.saveAsTable("products_table")

employee_delta_path = "/Workspace/Shared/employee_table"
product_delta_path = "/Workspace/Shared/product_table"

spark.sql(f"CREATE TABLE IF NOT EXISTS employee_delta USING DELTA LOCATION '{employee_delta_path}'")
spark.sql(f"CREATE TABLE IF NOT EXISTS product_delta USING DELTA LOCATION '{product_delta_path}'")

# Data modification

# Increase the salary by 5% for all employees in the IT department.
spark.sql("update employee_delta set Salary = Salary * 1.05 where Department = 'IT'")

#  Delete products where the stock is less than 40.
spark.sql("delete from product_delta where Stock < 40")

# Time Travel 

print("Products before delete operation")
spark.sql("SELECT * FROM product_delta VERSION AS OF 0").show()

print("Employees before update operation")
spark.sql("SELECT * FROM employee_delta VERSION AS OF 0").show()

# Query Delta Tables

# Query employees in finance department
spark.sql("SELECT * FROM employee_delta WHERE Department = 'Finance'").show()

# Query Electronics products with price > 500
spark.sql("SELECT * FROM product_delta WHERE Category = 'Electronics' AND Price > 500").show()






Products before delete operation
+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------+-----+---------+-----------+-----+
|Electronics| 1200|      101|     Laptop|   35|
|Electronics|  800|      102| Smartphone|   80|
|  Furniture|  150|      103| Desk Chair|   60|
|Electronics|  300|      104|    Monitor|   45|
|  Furniture|  350|      105|       Desk|   25|
+-----------+-----+---------+-----------+-----+

Employees before update operation
+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1001|     John Doe|        HR| 2021-01-15| 55000|
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-1